In [ ]:
# mp_report2 사용시 content와 ngram 열에 리스트가 str으로 변환되어 들어감.
# 특수 문자 제거과정 거쳐 mp_clean 생성 
import pandas as pd
import re

mp_clean = pd.read_csv('mp_report2.csv')
mp_clean

for i in range(len(mp_sent)):
    try: 
        mp_clean['content'][i] = re.sub(r"\(|\)|\[|\]|\'|-|｢|｣|•", "", mp_clean['content'][i])
        mp_clean['ngram'][i] = re.sub(r"\[|\]|\'", "", mp_clean['ngram'][i])
        print(i)
    except:
        print("failed")
        pass
    

mp_clean.to_csv('mp_clean.csv', index=False)
mp_clean


# I. 정제된 금통위의사록 데이터프레임 불러오기

In [ ]:
import csv
import pandas as pd
from ekonlpy.sentiment import MPCK

mpck = MPCK()
mp_clean = pd.read_csv('mp_clean.csv') # ngra 컬럼 사용x content만 추출
mp_clean

# II. 문장별 톤 구하기

## 1. 문장 별 ngram 구하기

In [ ]:
mp_total = []
for i in range(len(mp_clean)):
    mp_part = []
    for j in range(len(mp_clean['content'][i].split('.'))):
        text = mp_clean['content'][i].split('.')[j]
        tokens = mpck.tokenize(text) #토큰 리스트
        ngrams = mpck.ngramize(tokens) #엔그램 리스트
        ngram_list = []
        for k in range(len(tokens)):
            ngram_list.append(tokens[k])
        for l in range(len(ngrams)):
            ngram_list.append(ngrams[l])
        mp_part.append(ngram_list)
    mp_total.append(mp_part)
mp_total

## 2. 문장 별 어조 구하기 

In [ ]:
#각 문장의 어조 csv추출
neg_dic = pd.read_csv('close_neg_dic.csv', encoding = 'utf-8')
pos_dic = pd.read_csv('close_pos_dic.csv', encoding = 'utf-8')
#부정사전 리스트(금리 인하, Dovish)
negative_list = []
for i in range(len(neg_dic)):
    negative_list.append(neg_dic['ngram'][i])
#긍정사전 리스트(금리 인상, Hawkish)
positive_list = []
for i in range(len(pos_dic)):
    positive_list.append(pos_dic['ngram'][i])
#어조 분석
tone_list = [] #각 의사록별 톤 리스트 도출
for i in range(len(mp_total)): #각 의사록
    negative_sentence = 0
    positive_sentence = 0
    for j in range(len(mp_total[i])): #각 문장 -> 문장의 sentence_tone을 도출
        negative_word = 0
        positive_word = 0
        for k in range(len(mp_total[i][j])): #각 문장의 단어들
            if mp_total[i][j][k] in negative_list:
                negative_word += 1
            elif mp_total[i][j][k] in positive_list:
                positive_word += 1
            else:
                pass
        try:
            sentence_tone = (positive_word - negative_word) / (positive_word + negative_word)
        except:
            sentence_tone = 0
        if sentence_tone > 0:
            positive_sentence += 1
        elif sentence_tone < 0:
            negative_sentence += 1
        else:
            pass
    try:
        document_tone = (positive_sentence - negative_sentence) / (positive_sentence + negative_sentence)
        tone_list.append(document_tone)
    except:
        document_tone = 0  #각 의사록별 document_tone을 도출
        tone_list.append(document_tone)

In [ ]:
mp_tone = pd.DataFrame(index=range(0,len(mp_clean)), columns=['date','TONE-Doc'])
                       
mp_tone['date'] = mp_clean['date']
mp_tone['TONE-Doc'] = tone_list

mp_tone

In [ ]:
# 어조 데이터프레임에 기준금리 열 추가
base_rate = pd.read_csv("base_rate.csv", encoding = "utf-8")
mp_tone = pd.merge(mp_tone,base_rate, on='date',how='left')
mp_tone

In [ ]:
# 의사록 어조와 기준금리 간의 상관관계 측정
corr = mp_tone.corr()
print(corr)

# III. get score(논문 저자들이 사용한 함수)와 비교

In [ ]:
from ekonlpy.sentiment import MPKO
mpko = MPKO()

#각 문장의 어조 csv추출
neg_dic = pd.read_csv('close_neg_dic.csv', encoding = 'utf-8')
pos_dic = pd.read_csv('close_pos_dic.csv', encoding = 'utf-8')
#부정사전 리스트(금리 인하, Dovish)
negative_list = []
for i in range(len(neg_dic)):
    negative_list.append(neg_dic['ngram'][i])
#긍정사전 리스트(금리 인상, Hawkish)
positive_list = []
for i in range(len(pos_dic)):
    positive_list.append(pos_dic['ngram'][i])
#어조 분석
tone_list2 = [] #각 의사록별 톤 리스트 도출
for i in range(len(mp_total)): #각 의사록
    negative_sentence = 0
    positive_sentence = 0
    for j in range(len(mp_total[i])): #각 문장 -> 문장의 sentence_tone을 도출
        text = mp_total[i][j]
        tokens = mpko.tokenize(text)  
        sentence_tone = mpko.get_score(tokens)['Polarity']
        if sentence_tone > 0:
            positive_sentence += 1
        elif sentence_tone < 0:
            negative_sentence += 1
        else:
            pass
    try:
        document_tone = (positive_sentence - negative_sentence) / (positive_sentence + negative_sentence)
        tone_list2.append(document_tone)
    except:
        document_tone = 0  #각 의사록별 document_tone을 도출
        tone_list2.append(document_tone)



In [ ]:
mp_tone2 = pd.DataFrame(index=range(0,len(mp_clean)), columns=['date','TONE-Doc'])                      
mp_tone2['date'] = mp_clean['date']
mp_tone2['TONE-Doc'] = tone_list2
mp_tone2 = pd.merge(mp_tone2,base_rate, on='date',how='left')
mp_tone2

In [ ]:
corr = mp_tone2.corr()
print(corr)